# Load, select, clean json data 

In [2]:
import json 
import re
import os

In [ ]:
file_name = "camera-screen-guards.json"
base_name = os.path.splitext(os.path.basename(file_name))[0]


In [4]:
with open(file_name,'r',encoding="utf-8") as f:
    data = json.load(f)

In [5]:
item_list = data["mods"]["listItems"]

In [6]:
type(item_list)

list

In [7]:
def chunking_each_doc(doc,chunk_size = 128):
    start = 0
    doc = json.dumps(doc)
    tokens = re.findall(r'\w+|[{}[\]:,",]',doc)
    chunks = []
    while start <= len(tokens):
        end = min(start+chunk_size,len(tokens))
        chunk = " ".join(tokens[start:end])
        chunks.append(chunk)
        start+=chunk_size
    return chunks

In [8]:
all_chunks = []
for item in item_list:
    chunks_of_each_doc= chunking_each_doc(item)
    all_chunks.append(chunks_of_each_doc)


In [9]:
len(all_chunks)

40

In [10]:
all_chunks[0][0]

'{ " name " : " Redmi Note 14 5g Camera Protector Mettalic Ring Glass " , " nid " : " 361653530 " , " itemId " : " 361653530 " , " icons " : [ { " domClass " : " 150565 " , " type " : " img " , " group " : " 6 " , " showType " : " 0 " } , { " domClass " : " 175175 " , " type " : " img " , " group " : " 3 " , " showType " : " 0 " } ] , " image " : " https : static 01 daraz com np p 59a3b411d874aeeea2f115c998b18143 jpg " , " isSmartImage " :'

In [11]:
all_chunks[0]

['{ " name " : " Redmi Note 14 5g Camera Protector Mettalic Ring Glass " , " nid " : " 361653530 " , " itemId " : " 361653530 " , " icons " : [ { " domClass " : " 150565 " , " type " : " img " , " group " : " 6 " , " showType " : " 0 " } , { " domClass " : " 175175 " , " type " : " img " , " group " : " 3 " , " showType " : " 0 " } ] , " image " : " https : static 01 daraz com np p 59a3b411d874aeeea2f115c998b18143 jpg " , " isSmartImage " :',
 'false , " utLogMap " : { " srp_name " : " LazadaMainSrp " , " x_object_type " : " item " , " src " : " organic " , " trafficType " : " organic " , " x_sku_ids " : " 1592835298 " , " x_item_ids " : " 361653530 " , " iconList " : " 150565 175175 " , " SN " : " feb7c324dba78761f11a6be643f46dc6 " , " current_price " : " 239 " , " x_object_id " : " 361653530 " } , " originalPriceShow " : " " , " priceShow " : " Rs 239 " , " ratingScore " : " 4 4 " , " review " : " 5 " , " location " : " Bagmati',
 'Province " , " description " : [ ] , " thumbs " : [ 

In [12]:
type(all_chunks)

list

# Qdrant Operation for Hybrid Search

In [ ]:
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer

In [15]:
dense_model_name = "sentence-transformers/all-MiniLM-L6-v2"
sparse_model_name = "prithivida/Splade_PP_en_v1"

In [16]:
client = QdrantClient(url = "http://localhost:6333")

In [17]:
if not client.collection_exists("hybrid_search_daraz"):
    client.create_collection(
        collection_name = "hybrid_search_daraz",
        vectors_config = {
            "dense" : models.VectorParams(size = client.get_embedding_size(dense_model_name), distance = models.Distance.COSINE)

        },
        sparse_vectors_config= {
            "sparse" : models.SparseVectorParams()
        }
    )

In [31]:
# creation of embeddings

#dense vectors and sparse vectors of the same document and chunks
final_dense_vector = []
final_sparse_vector = []
for i in range(len(all_chunks)):
    for j in range(len(all_chunks[i])):
        dense_embeddings = models.Document(text = all_chunks[i][j],model = dense_model_name)
        final_dense_vector.append(dense_embeddings)
        sparse_embeddings = models.Document(text = all_chunks[i][j],model = sparse_model_name)
        final_sparse_vector.append(sparse_embeddings)




In [24]:
from sentence_transformers import SentenceTransformer

In [25]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [23]:
final_dense_vector[0]

Document(text='{ " name " : " Redmi Note 14 5g Camera Protector Mettalic Ring Glass " , " nid " : " 361653530 " , " itemId " : " 361653530 " , " icons " : [ { " domClass " : " 150565 " , " type " : " img " , " group " : " 6 " , " showType " : " 0 " } , { " domClass " : " 175175 " , " type " : " img " , " group " : " 3 " , " showType " : " 0 " } ] , " image " : " https : static 01 daraz com np p 59a3b411d874aeeea2f115c998b18143 jpg " , " isSmartImage " :', model='sentence-transformers/all-MiniLM-L6-v2', options=None)

In [ ]:
final_dense_vector

262

In [ ]:
final_sparse_vector

In [34]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "doc_id",
    field_schema = "integer"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "chunk_id",
    field_schema = "integer"
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [37]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "file_name",
    field_schema = "keyword"
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
offset = 0
info = client.get_collection(collection_name = "hybrid_search_daraz")
file_number = 0
client.scroll(
    collection_name = "hybrid_search_daraz",
    field_name = "doc_id",
    field_schema = "integer"
)

AssertionError: Unknown arguments: ['field_name', 'field_schema']